# QCustom model Training using CNRPark-EXT DS1
Date: 2024-07-15

Author: Ziad Tamim

Discription:
Thsi Script includes the training of the QCustom model. This includes model structure, Quantisation  and knowledge disilation.  

Inputs:
* Dataset

Outputs:
* Custom model (with no quanisation)
* QCustom model (quanized Custom model)



## Load dataset

In [ ]:
# Path to files
traing_path = 'C:/Users/ziadt/Desktop/Projects/MSc Project implimintation/Datasets/CNP-EXT - Dataset/CNR-EXT-Patches-150x150/LABELS/train.txt'
test_path = 'C:/Users/ziadt/Desktop/Projects/MSc Project implimintation/Datasets/CNP-EXT - Dataset/CNR-EXT-Patches-150x150/LABELS/train.txt'
val_path = 'C:/Users/ziadt/Desktop/Projects/MSc Project implimintation/Datasets/CNP-EXT - Dataset/CNR-EXT-Patches-150x150/LABELS/train.txt'
base_path = 'C:/Users/ziadt/Desktop/Projects/MSc Project implimintation/Datasets/CNP-EXT - Dataset/CNR-EXT-Patches-150x150/PATCHES'  # Ensure this is the correct base path to your images


# Open files
train_file = open(traing_path, 'r')
test_file = open(test_path, 'r')
val_file = open(val_path, 'r')

# Read lines
train_lines = train_file.readlines()
test_lines = test_file.readlines()
val_lines = val_file.readlines()
train_file.close()
test_file.close()
val_file.close()

# Lists to store images and labels
train_images = []
train_labels = []
test_images = []
test_labels = []
val_images = []
val_labels = []

# Split lines
for line in train_lines:
    line = line.split(' ')
    train_images.append(line[0])
    train_labels.append(int(line[1]))

for line in test_lines:
    line = line.split(' ')
    test_images.append(line[0])
    test_labels.append(int(line[1]))

for line in val_lines:
    line = line.split(' ')
    val_images.append(line[0])
    val_labels.append(int(line[1]))

# Print info
print('Train images:', len(train_images))
print('Train labels:', len(train_labels))
print('Test images:', len(test_images))
print('Test labels:', len(test_labels)) 
print('Val images:', len(val_images))
print('Val labels:', len(val_labels))
print('Total images:', len(train_images) + len(test_images) + len(val_images))
print('\n')
print('train image: ', train_images[0])
print('train label: ', train_labels[0])


View data for inspection

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# Assuming train_images[0] contains the relative path from the notebook's current directory
n = 1
image_path = train_images[n]
full_image_path = os.path.join(base_path, image_path)

# Check if the file exists
if os.path.exists(full_image_path):
    image = cv2.imread(full_image_path)
    print(full_image_path)
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.show()
        print('Label:', train_labels[n])
    else:
        print("Failed to load the image. The file may be corrupted.")
else:
    print("Image path does not exist:", full_image_path)


In [ ]:
# discription: This code is used to load the images and labels from the dataset and create a dataset object that can be used to train the model.
# The dataset is then preprocessed for performance using the tf.data API. The images are resized to 150x150 pixels and the dataset is shuffled,
# batched, and prefetched for performance. 
# The dataset is then ready to be used for training a model.

import tensorflow as tf
import os


def parse_function(filename, label):
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [150, 150]) # change size to 227x227 for AlexNet
    # Ensure label is also returned correctly
    return image_resized, label

def create_dataset(filepaths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(filepaths)
    labels_ds = tf.data.Dataset.from_tensor_slices(labels)
    # Use dataset map with both filename and label
    full_dataset = tf.data.Dataset.zip((path_ds, labels_ds))
    image_label_ds = full_dataset.map(parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return image_label_ds

# Full file paths for the images
train_paths = [os.path.join(base_path, img) for img in train_images]
test_paths = [os.path.join(base_path, img) for img in test_images]
val_paths = [os.path.join(base_path, img) for img in val_images]

print(train_paths[0])
# Create datasets
train_ds = create_dataset(train_paths, train_labels)
test_ds = create_dataset(test_paths, test_labels)
val_ds = create_dataset(val_paths, val_labels)

# Configure for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.shuffle(1000).batch(32).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.batch(32).prefetch(buffer_size=AUTOTUNE)


## The Custom model structure (Student model)
### Discription
This code defines the Custom model (Student) use in a knowledge distillation process. The model is intended to be trained by learning from a pre-trained "teacher" model. The student model takes 150x150x3 input images and passes them through a series of layers, including initial convolutional layers, multiple MobileNet blocks, and a global average pooling layer. The final output layer uses a sigmoid activation function to produce a single probability score for binary classification (Occupied or Free). The model's compact architecture makes it suitable for distillation, where it will be trained to mimic the behavior of a more complex teacher model.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, DepthwiseConv2D, Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, GlobalAveragePooling2D, Dense
from tensorflow.keras import Model


# MobileNet block
def mobilnet_block(x, filters, strides): # this function is used to create a mobilenet block consisting of depthwise convolution followed by pointwise convolution
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x) # depthwise convolution
    x = BatchNormalization()(x) 
    x = ReLU()(x)
    
    x = Conv2D(filters=filters, kernel_size=1, strides=1)(x) # pointwise convolution
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x # return the output of the block

# Input specification for the model
input = Input(shape=(150, 150, 3))  # image input size is 150x150x3
x = Conv2D(filters=16, kernel_size=3, strides=2, padding='same')(input) # first layer conv2d
x = BatchNormalization()(x) # first layer batch normalization
x = ReLU()(x) # first layer ReLU

# Main part of the model
x = mobilnet_block(x, filters=5, strides=1) # second layer mobilenet block
x = mobilnet_block(x, filters=5, strides=2) # third layer mobilenet block
x = mobilnet_block(x, filters=12, strides=1) # fourth layer mobilenet block
x = mobilnet_block(x, filters=12, strides=2) # fifth layer mobilenet block
x = mobilnet_block(x, filters=24, strides=1) # sixth layer mobilenet block
x = mobilnet_block(x, filters=24, strides=2) # seventh layer mobilenet block
x = mobilnet_block(x, filters=24, strides=2) # eighth layer mobilenet block

# Adjusting for binary classification
x = GlobalAveragePooling2D()(x)  # Changed from AvgPool2D to GlobalAveragePooling2D
output = Dense(units=1, activation='sigmoid')(x)  # Changed to one unit with sigmoid activation

# Create the model
student = Model(inputs=input, outputs=output) # create the model 
student.summary()
student.input_shape


## Resnet50 model (Teacher model)
This code sets up the teacher model for knowledge distillation using a pre-trained ResNet50 architecture. The ResNet50 model's layers are frozen to retain the pre-trained ImageNet weights. A global average pooling layer and a sigmoid-activated dense layer are added on top to adapt the model for binary classification. This model will serve as the teacher in the knowledge distillation process.

In [ ]:
# train on resnet
from tensorflow.keras.applications import ResNet50

# load the model
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
resnet.summary()

# Freeze the layers
for layer in resnet.layers:
    layer.trainable = False

# Add the top layers
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)
teacher = Model(resnet.input, x)

## Distillation Model Setup
This code defines the Distiller class, a custom TensorFlow model designed to facilitate knowledge distillation. The class combines a teacher and student model, allowing the student to learn from both the true labels and the teacher's predictions. The distillation process is controlled by parameters like alpha (balancing student and distillation loss) and temperature (smoothing the teacher's predictions). The Distiller is then instantiated, compiled with binary cross-entropy for the student loss, and Kullback-Leibler divergence for the distillation loss, and is ready for training.

In [ ]:

class Distiller(tf.keras.Model): # this class is used to create a distiller model that will be used to train the student model
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile( # this function is used to compile the model
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1, # alpha is the weight given to the student loss
        temperature=3, # temperature is used to soften the predictions
    ):
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics) # compile the model with the optimizer and metrics
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data): # this function is used to train the model using the distillation loss
        x, y = data
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            student_predictions = self.student(x, training=True)
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss # calculate the loss using the distillation loss

        gradients = tape.gradient(loss, self.student.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.student.trainable_variables))
        self.compiled_metrics.update_state(y, student_predictions)
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss, "distillation_loss": distillation_loss}) # update the results with the student loss and distillation loss
        return results

    #Test 2
    def test_step(self, data): # this function is used to test the model using the distillation loss
        x, y = data
        student_predictions = self(x, training=False)
        student_loss = self.student_loss_fn(y, student_predictions)

        # Update metrics
        self.compiled_metrics.update_state(y, student_predictions)

        # Collect metrics results
        results = {m.name: m.result() for m in self.metrics}
        results['loss'] = student_loss
        return results

    def call(self, inputs, training=False): # this function is used to call the model
        if training:
            return self.student(inputs, training=True)
        else:
            return self.student(inputs, training=False)

# Create the distiller instance again and compile it
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
    student_loss_fn=tf.keras.losses.BinaryCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10
)

## Training

In [ ]:
# tensorboard initialization
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard

path_to_logs = "C:/Users/ziadt/Desktop/Projects/MSc Project implimintation/Model Training/CustomeNEt/training supporting material/logs/fit/"

# Create a TensorBoard callback
log_dir = path_to_logs + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Train the distiller
history = distiller.fit(train_ds, epochs=1, batch_size=32, validation_data=val_ds, callbacks=[tensorboard_callback])


In [ ]:
# accuracy perfomance on the test set
distiller.evaluate(test_ds)



## Save the Custom model without Quantization

In [ ]:
# save model
student.save('C:/Users/ziadt/Desktop/Projects/MSc Project implimintation/Model Training/CustomeNEt/training supporting material/student_model.h5')


In [ ]:
# load model
from tensorflow.keras.models import load_model
model = load_model('C:/Users/ziadt/Desktop/Projects/MSc Project implimintation/Model Training/CustomeNEt/training supporting material/student_model.h5')

## Post Training Interger Quantization


In [ ]:
# full integer quantization
converter = tf.lite.TFLiteConverter.from_keras_model(distiller.student)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model to disk
open('QCustom_D1_V1.tflite', 'wb').write(tflite_model)


## Inference speed of the QCustom model on a single slot

In [ ]:
import numpy as np
import time

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# test data
n = 0
image_path = test_images[n]
full_image_path = os.path.join(base_path, image_path)

image = cv2.imread(full_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (150, 150))
image = np.expand_dims(image, axis=0)
image = image / 255.0
image = tf.convert_to_tensor(image, dtype=tf.float32)


# Test the model on random input data.
times = []
for i in range(100):
    start_time = time.time()
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    end_time = time.time()
    times.append(end_time - start_time)

print('Average inference time for 100 images with quantized model:', np.mean(times))

